# `state_dict`

In [1]:
import torch
import torch.nn as nn

## `save`

[torch.save](https://pytorch.org/docs/stable/generated/torch.save.html)

テンソルやモデルの保存。拡張子には`.pt`や`.pth`が基本使われる。

どちらを使うべき、みたいな話は多分なくて、本当にどっちでも良さそう。公式にも

> A common PyTorch convention is to save models using either a `.pt` or `.pth` file extension.

って書いてある。ただ`.pt`はzopeというwebフレームワークで用いるテンプレートファイルにも使われるようで、一応重複を避けようという気持ちで私は`.pth`を使っている。`.pt`にするとvscodeでテキストファイルのアイコンなんかが出て気持ち悪かったりもする。でもPyTorch公式のexampleでは`.pt`が基本使われているし、copilotが提示するコードも基本`.pt`なので、`.pt`の方が一般的なのかもしれない。

In [2]:
x = torch.tensor([1, 2, 3])
torch.save(x, "data.pth")

In [3]:
model = nn.Linear(3, 2)
torch.save(model, "model.pth") # この使い方はしない方がいい（詳細は後で）

別にテンソルやモデルでなくてもいい。任意のオブジェクトを保存することができる。pickleと同じ。

In [4]:
class MyObject:
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c

    def __repr__(self):
        return f"MyObject(a={self.a}, b={self.b}, c={self.c})"

obj = MyObject(1, 2, 3)
torch.save(obj, "obj.pth")

ただ、この使い方もしない方がいい。普通にpickle使おう。

## `load`

[torch.load](https://pytorch.org/docs/stable/generated/torch.load.html)

保存したファイルを読み込む。

In [5]:
data = torch.load("data.pth")
data

/tmp/ipykernel_1644820/1216731273.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load("data.pth")


tensor([1, 2, 3])

In [6]:
model = torch.load("model.pth")
model

/tmp/ipykernel_1644820/3675544202.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("model.pth")


Linear(in_features=3, out_features=2, bias=True)

In [7]:
obj = torch.load("obj.pth")
obj

/tmp/ipykernel_1644820/2542391371.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  obj = torch.load("obj.pth")


MyObject(a=1, b=2, c=3)

ファイルは全て読み込めたけど、なんか警告出たな。

「今は`weights_only=False`がデフォルトだけど、近いうちに`True`にするから注意してや」って言ってるね。

In [8]:
"ちなみに今のバージョンは" + torch.__version__

'ちなみに今のバージョンは2.4.0+cu121'

`weights_only=True`は、読み込めるオブジェクトをテンソルと辞書（+α）だけにするということ。

モデルの保存には二つのやり方があって、一つはさっきやったようにモデルをそのまま保存するやり方。もう一つはモデルの重みだけを保存するやり方で、`model.state_dict()`を使う（詳細は後程）。こうすると重みだけが辞書として保存される。

`weights_only=True`だと前者のやり方が使えなくなる。これをデフォルトにすると言っているので、今後は前者を使いたい場合に`weights_only=False`を明示的に指定する必要がある。まあ余程のことがない限り後者を使った方がいいと思うけど。わざわざデフォルト値を変えるということは、そこで問題が起きやすいということなので、大人しく従った方がいい。実際にモデルの保存には`state_dict()`を使うことが推奨されている。

> Instead of saving a module directly, for compatibility reasons it is recommended to instead save only its state dict.

https://pytorch.org/docs/stable/notes/serialization.html#saving-and-loading-torch-nn-modules

## `state_dict`

[What is a state_dict in PyTorch](https://pytorch.org/tutorials/recipes/recipes/what_is_state_dict.html)

モデルパラメータを保存するための辞書。`model.state_dict()`で取得できる。

In [9]:
model = nn.Linear(3, 2)
model.state_dict()

OrderedDict([('weight',
              tensor([[-0.2987,  0.3579,  0.4962],
                      [-0.4645,  0.2501, -0.3249]])),
             ('bias', tensor([-0.4080,  0.0948]))])

キーが属性名、値はテンソル。

`OrderedDict`というオブジェクトで、通常の辞書とは違い、順序が保持される。`collections`という標準モジュールの機能。

In [10]:
type(model.state_dict())

collections.OrderedDict

In [11]:
from collections import OrderedDict

### モデルの保存・読み込み

モデルを保存するときはこの辞書を保存することが推奨されている。保存したいのはパラメータだけなのでこれでよい。

In [12]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(3, 2)
        self.fc2 = nn.Linear(2, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

model = MyModel()
state_dict = model.state_dict()
torch.save(state_dict, "model.pth")

`state_dict`を読み込むときは`load_state_dict()`を使う。

In [13]:
model = MyModel()
state_dict = torch.load("model.pth", weights_only=True)
model.load_state_dict(state_dict)

<All keys matched successfully>

キーが足りなかったり余分なキーがあったりするとエラーが出る。

In [14]:
state_dict_incorrect = state_dict.copy()
state_dict_incorrect.pop("fc2.weight")
state_dict_incorrect["fc1234.weight"] = torch.tensor([1, 2, 3, 4])
try:
    model.load_state_dict(state_dict_incorrect)
except Exception as e:
    print(e)

Error(s) in loading state_dict for MyModel:
	Missing key(s) in state_dict: "fc2.weight". 
	Unexpected key(s) in state_dict: "fc1234.weight". 


`strict=False`を指定すると勝手に無視して読み込んでくれる。

In [15]:
model.load_state_dict(state_dict_incorrect, strict=False)

_IncompatibleKeys(missing_keys=['fc2.weight'], unexpected_keys=['fc1234.weight'])

正しいキーで間違った値を入れると、いかなる場合でもエラーを出してくれる。

In [16]:
state_dict_incorrect = state_dict.copy()
state_dict_incorrect["fc2.weight"] = torch.tensor([99, 99, 99, 99, 99])
try:
    model.load_state_dict(state_dict_incorrect, strict=False)
except Exception as e:
    print(e)

Error(s) in loading state_dict for MyModel:
	size mismatch for fc2.weight: copying a param with shape torch.Size([5]) from checkpoint, the shape in current model is torch.Size([1, 2]).


読み込む辞書のキーと同じ名前（属性）のパラメータをモデルが保有している必要がある。だから、例えば変数名を変えたりしてもエラーが出る。

In [17]:
class MyModel2(nn.Module):
    def __init__(self):
        super().__init__()
        # self.fc1 = nn.Linear(3, 2)
        self.fc_1 = nn.Linear(3, 2) # アンダースコアを付けた
        self.fc2 = nn.Linear(2, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc_1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

model = MyModel2()
try:
    model.load_state_dict(state_dict)
except Exception as e:
    print(e)

Error(s) in loading state_dict for MyModel2:
	Missing key(s) in state_dict: "fc_1.weight", "fc_1.bias". 
	Unexpected key(s) in state_dict: "fc1.weight", "fc1.bias". 


### デバイス

`torch.Tensor`はデバイスの情報を持つ。モデルパラメータも`torch.Tensor`なので、当然、モデルパラメータもデバイスの情報を持つ。

`state_dict`で得たパラメータにもデバイスの情報が乗っている。`state_dict()`を呼び出す際のモデルのデバイスと同じものが乗る。

In [18]:
cuda = torch.device("cuda")
model = MyModel()
model.to(cuda)
state_dict = model.state_dict()
torch.save(state_dict, "model.pth")
state_dict

OrderedDict([('fc1.weight',
              tensor([[ 0.4732,  0.4301,  0.1034],
                      [ 0.3597,  0.1872, -0.0266]], device='cuda:0')),
             ('fc1.bias', tensor([-0.2967,  0.3656], device='cuda:0')),
             ('fc2.weight', tensor([[ 0.5535, -0.5524]], device='cuda:0')),
             ('fc2.bias', tensor([0.1616], device='cuda:0'))])

これを読み込むと、当然同じデバイスでパラメータが読み込まれる。

In [19]:
state_dict = torch.load("model.pth", weights_only=True)
state_dict

OrderedDict([('fc1.weight',
              tensor([[ 0.4732,  0.4301,  0.1034],
                      [ 0.3597,  0.1872, -0.0266]], device='cuda:0')),
             ('fc1.bias', tensor([-0.2967,  0.3656], device='cuda:0')),
             ('fc2.weight', tensor([[ 0.5535, -0.5524]], device='cuda:0')),
             ('fc2.bias', tensor([0.1616], device='cuda:0'))])

`map_location`で読み込む際のデバイスを指定できる。

In [20]:
cpu = torch.device("cpu")
state_dict = torch.load("model.pth", weights_only=True, map_location=cpu)
state_dict

OrderedDict([('fc1.weight',
              tensor([[ 0.4732,  0.4301,  0.1034],
                      [ 0.3597,  0.1872, -0.0266]])),
             ('fc1.bias', tensor([-0.2967,  0.3656])),
             ('fc2.weight', tensor([[ 0.5535, -0.5524]])),
             ('fc2.bias', tensor([0.1616]))])

モデルに読み込む場合は、そのモデルのデバイスに合わせて読み込まれる。

In [21]:
model = MyModel() # CPUに作成
state_dict = torch.load("model.pth", weights_only=True, map_location=cuda) # GPUに読み込み
model.load_state_dict(state_dict)
model.fc1.weight.device # CPUに読み込まれる

device(type='cpu')

In [22]:
# モデルがGPUにあるときはGPUに読み込まれる
model = MyModel()
model.to(cuda)
model.load_state_dict(state_dict)
model.fc1.weight.device

device(type='cuda', index=0)

### `torch.compile`

`torch.compile`をした後はキーに注意しないといけない。

In [23]:
model = torch.compile(model)
model

OptimizedModule(
  (_orig_mod): MyModel(
    (fc1): Linear(in_features=3, out_features=2, bias=True)
    (fc2): Linear(in_features=2, out_features=1, bias=True)
    (relu): ReLU()
    (sigmoid): Sigmoid()
  )
)

与えたモデルが`_orig_mod`としてまとめられている。つまりこれの`state_dict`は

In [24]:
model.state_dict()

OrderedDict([('_orig_mod.fc1.weight',
              tensor([[ 0.4732,  0.4301,  0.1034],
                      [ 0.3597,  0.1872, -0.0266]], device='cuda:0')),
             ('_orig_mod.fc1.bias',
              tensor([-0.2967,  0.3656], device='cuda:0')),
             ('_orig_mod.fc2.weight',
              tensor([[ 0.5535, -0.5524]], device='cuda:0')),
             ('_orig_mod.fc2.bias', tensor([0.1616], device='cuda:0'))])

こうなる。全てのキーに`_orig_mod.`がついている。これを読み込む場合、compile済みのモデルに読み込む必要がある。それは少々不便なので、

In [25]:
model._orig_mod.state_dict()

OrderedDict([('fc1.weight',
              tensor([[ 0.4732,  0.4301,  0.1034],
                      [ 0.3597,  0.1872, -0.0266]], device='cuda:0')),
             ('fc1.bias', tensor([-0.2967,  0.3656], device='cuda:0')),
             ('fc2.weight', tensor([[ 0.5535, -0.5524]], device='cuda:0')),
             ('fc2.bias', tensor([0.1616], device='cuda:0'))])

とする。こうすれば`_orig_mod.`がつかない。

ただこれはcompile済みのモデルにしか使えない。compile前でも後でも良いようにするには以下の通り。

In [26]:
getattr(model, "_orig_mod", model).state_dict()

OrderedDict([('fc1.weight',
              tensor([[ 0.4732,  0.4301,  0.1034],
                      [ 0.3597,  0.1872, -0.0266]], device='cuda:0')),
             ('fc1.bias', tensor([-0.2967,  0.3656], device='cuda:0')),
             ('fc2.weight', tensor([[ 0.5535, -0.5524]], device='cuda:0')),
             ('fc2.bias', tensor([0.1616], device='cuda:0'))])

`_orig_mod`属性を持っていればそれ、なければそのまま。